In [1]:
import urllib, json
import pandas as pd
import numpy as np
import csv

In [2]:
# -----------
# input block
# -----------

# -----------------------------------
# set one of the following to be True
# -----------------------------------
i_train_input       = 0 
i_test_input        = 0
i_application_input = 1

# ------------------------------------
# Select States(with 2 audits) to test
# ------------------------------------
states_test=['IRN','DNK','BGR','UZB','SEN','BWA'] # States to be removed in the training list

# ---------------------------
if i_application_input:
    print('Please give the 3 letter State code:\n')
states_to_predict=['PRK','TLS','COM','ZWE','TUN','TKM','OMN','SAU','BOL','COL','CIV','SEN']
year_to_predict= 2019

# ----------------
# defaut parameter
# ----------------
row = ['EI','dYear','GDP','ce6','ce3','ce2'] # accident, ICVM
nb_inputs = len(row)
nb_outputs = 3  # decrease (more than 10%) [0 0 1], stable [0 1 0], increase (more than 10%) [1 0 0]


Please give the 3 letter State code:



In [3]:
#--------------------------------------------------------------------
# first create list of States with two Audits and Year of each Audit
#--------------------------------------------------------------------
url='https://v4p4sz5ijk.execute-api.us-east-1.amazonaws.com/anbdata/states/usoap/activity-list?api_key=00cfdca0-7039-11e9-9730-617277d0f137&format=json'
response = urllib.request.urlopen(url)
data = json.loads(response.read().decode("utf-8"))
df=pd.DataFrame(data) #print(df.columns)

df_grp_act_St    = df.groupby(['Activity','isFinal'])['State'].apply(list)
df_grp_act_Y     = df.groupby(['Activity','State','isFinal'])['Year'].apply(list)
df_grp_act_EI    = df.groupby(['Activity','State',])['overall'].apply(list)

#---------------
act1='CMA Audit'
status=1 # True: final report
#---------------
states_all =[] # list of all States
states_2 =[] # list of States with at least two CMA Audit

for etat in df_grp_act_St[act1,status]:
    states_all.append(etat)
    if len(df_grp_act_Y[act1,etat,status])>=2:
        states_2.append(etat)
        
states_2.remove('SMR')
states_2.remove('SMR') # no economy data after 2008
A=set(states_2)
states_ls_2 = list(A)
B=set(states_all)
states_ls_all = list(B)
print(len(states_ls_2),'States have at least 2 EI data:\n',states_ls_2, ' among', len(states_ls_all))

if i_train_input:
    print('\ncreating inputs for the training.......')
    for St in states_test:
        states_ls_2.remove(St)
        
if i_test_input:
    print('\ncreating inputs for the validation test.......')
    print('\nValidation States:', states_test)
    states_ls_2 = states_test
if i_application_input:
    states_ls_2 =states_to_predict
    print('\nValidation States:', states_to_predict)
year_ls_2=np.zeros((len(states_ls_2),2))
mid_year_ls_2=np.zeros(len(states_ls_2))
dYear=np.zeros(len(states_ls_2))

#-----------------------------------------------
istate =0
for etat in states_ls_2:
    year_ls_2[istate,0]  = int(df_grp_act_Y[act1][etat][status][0])
    year_ls_2[istate,1]  = int(df_grp_act_Y[act1][etat][status][-1])
    if etat=='SEN' or etat=='COL':
        print(df_grp_act_Y[act1][etat][status])
        year_ls_2[istate,1]  = df_grp_act_Y[act1][etat][status][-2]
    if i_application_input:
         year_ls_2[istate,0] = year_to_predict
    mid_year_ls_2[istate]= np.int((year_ls_2[istate,0] + year_ls_2[istate,1])/2.0)
    dYear[istate]        = year_ls_2[istate,0] - year_ls_2[istate,1]
    istate+=1
print(year_ls_2.transpose())
print('middle year',mid_year_ls_2)
mean_dYear=int(np.mean(dYear))

50 States have at least 2 EI data:
 ['KGZ', 'BHS', 'NZL', 'ARM', 'TZA', 'MRT', 'MYS', 'EGY', 'GMB', 'IND', 'BGR', 'DEU', 'SEN', 'UZB', 'UKR', 'PHL', 'ISR', 'HND', 'MAR', 'ETH', 'PAN', 'NOR', 'PER', 'POL', 'THA', 'TJK', 'AUS', 'IRN', 'GTM', 'COL', 'DNK', 'KEN', 'MMR', 'LKA', 'BOL', 'BRA', 'KHM', 'LBN', 'AZE', 'SYC', 'ZAF', 'GRC', 'KWT', 'NGA', 'JOR', 'GEO', 'IDN', 'BWA', 'RUS', 'QAT']  among 187

Validation States: ['PRK', 'TLS', 'COM', 'ZWE', 'TUN', 'TKM', 'OMN', 'SAU', 'BOL', 'COL', 'CIV', 'SEN']
[2017, 2007]
[2019, 2016, 2006]
[[2019. 2019. 2019. 2019. 2019. 2019. 2019. 2019. 2019. 2019. 2019. 2019.]
 [2008. 2010. 2008. 2010. 2009. 2010. 2010. 2009. 2008. 2017. 2008. 2016.]]
middle year [2013. 2014. 2013. 2014. 2014. 2014. 2014. 2014. 2013. 2018. 2013. 2017.]


array([11.,  9., 11.,  9., 10.,  9.,  9., 10., 11.,  2., 11.,  3.])

In [5]:
#---------------------------------
# create input of initial EI 
#---------------------------------
nb_idx=0
input_matrix  = np.zeros((len(states_ls_2),nb_inputs))
result_matrix = np.zeros((len(states_ls_2),nb_outputs))

for in_para in row:
    if in_para=='EI': 
        current_EI_ave = 68.0
        nb_etat=0
        for etat in states_ls_2:
            dEI_temp= df_grp_act_EI[act1][etat][0] - df_grp_act_EI[act1][etat][-1]

            temp_array = df_grp_act_EI[act1][etat][-1]
            input_matrix[nb_etat,nb_idx] = (temp_array>=current_EI_ave)
            
            if i_train_input or i_test_input:
                if dEI_temp > 10.0:
                    result_matrix[nb_etat] = [1,0,0]
                elif dEI_temp < -10.0:
                    result_matrix[nb_etat] = [0,0,1]
                else:
                    result_matrix[nb_etat] = [0,1,0]
            nb_etat+=1

        nb_idx+=1

        #print('---------------\n input matrix:\n---------------\n',input_matrix.transpose())
        #print('---------------\n result vector:\n---------------\n',result_matrix.transpose())
        result_array=list(result_matrix)

    if in_para=='dYear':
        nb_etat=0
        for etat in states_ls_2:
            input_matrix[nb_etat,nb_idx] = (dYear[nb_etat]>=mean_dYear)
            nb_etat+=1
        nb_idx+=1
    
    if in_para=='GDP':
        url='https://v4p4sz5ijk.execute-api.us-east-1.amazonaws.com/anbdata/states/system/economic-stats?api_key=00cfdca0-7039-11e9-9730-617277d0f137&format=json'
        response = urllib.request.urlopen(url)
        data = json.loads(response.read().decode("utf-8"))
        df=pd.DataFrame(data)
        df.replace('', 0.0, inplace=True)

        df_GDP  = df.groupby(['Year','State',])['GDP'].apply(list)
        df_dGDP = df.groupby(['Year','State',])['GDP_growth_pc'].apply(list)
        df_corru= df.groupby(['Year','State',])['corruption_wgi'].apply(list)

        # for the computation of the global mean at each mid-year
        df_GDP_2  = df.groupby(['Year'])['GDP'].apply(list)
        df_dGDP_2 = df.groupby(['Year'])['GDP_growth_pc'].apply(list)
        df_corru_2= df.groupby(['Year'])['corruption_wgi'].apply(list)
         
        nb_etat=0
        for etat in states_ls_2:
            year = int(mid_year_ls_2[nb_etat])
            if year>2014:
                year = 2014 # no GDP data after 2014
            #print(df_GDP_2)
            print('year',year)
            ave_GDP_mid   = np.mean(df_GDP_2[year])
            ave_dGDP_mid  = np.mean(df_dGDP_2[year])
            ave_corru_mid = np.mean(df_corru_2[year])
            
            comp_array=[ave_GDP_mid]#,ave_dGDP_mid,ave_corru_mid]
            temp_array=[df_GDP[year,etat]]#,df_dGDP[year,etat],df_corru[year,etat]]
            
            input_matrix[nb_etat,nb_idx] = (temp_array>=comp_array)
            nb_etat+=1
           
        nb_idx+=1  
    if in_para=='ce6': # of first audit year
        url='https://v4p4sz5ijk.execute-api.us-east-1.amazonaws.com/anbdata/states/usoap/activity-list?api_key=00cfdca0-7039-11e9-9730-617277d0f137&format=json'
        response = urllib.request.urlopen(url)
        data = json.loads(response.read().decode("utf-8"))
        df=pd.DataFrame(data)
        df.replace(np.nan, 0.0, inplace=True)
        #print(df.columns)
        for i_ce in [in_para,]:
            df_CE_temp_0    = df.groupby(['Activity','isFinal',])[i_ce].apply(list)
            df_CE_ave_temp  = np.mean(df_CE_temp_0['CMA Audit',1])
            df_CE_temp_1    = df.groupby(['Activity','State','isFinal',])[i_ce].apply(list)
            nb_etat=0
            for etat in states_ls_2:
                comp_array3     = df_CE_ave_temp
                temp_array3     = df_CE_temp_1['CMA Audit',etat,1][0]
                input_matrix[nb_etat][nb_idx] = (temp_array3>=comp_array3)
                nb_etat+=1
            nb_idx+=1
            
    if in_para=='ce3': # of first audit year
        url='https://v4p4sz5ijk.execute-api.us-east-1.amazonaws.com/anbdata/states/usoap/activity-list?api_key=00cfdca0-7039-11e9-9730-617277d0f137&format=json'
        response = urllib.request.urlopen(url)
        data = json.loads(response.read().decode("utf-8"))
        df=pd.DataFrame(data)
        df.replace(np.nan, 0.0, inplace=True)
        #print(df.columns)
        for i_ce in [in_para,]:
            df_CE_temp_0    = df.groupby(['Activity','isFinal',])[i_ce].apply(list)
            df_CE_ave_temp  = np.mean(df_CE_temp_0['CMA Audit',1])
            df_CE_temp_1    = df.groupby(['Activity','State','isFinal',])[i_ce].apply(list)
            nb_etat=0
            for etat in states_ls_2:
                comp_array3     = df_CE_ave_temp
                temp_array3     = df_CE_temp_1['CMA Audit',etat,1][0]
                input_matrix[nb_etat][nb_idx] = (temp_array3>=comp_array3)
                nb_etat+=1
            nb_idx+=1
            
    if in_para=='ce2': # of first audit year
        url='https://v4p4sz5ijk.execute-api.us-east-1.amazonaws.com/anbdata/states/usoap/activity-list?api_key=00cfdca0-7039-11e9-9730-617277d0f137&format=json'
        response = urllib.request.urlopen(url)
        data = json.loads(response.read().decode("utf-8"))
        df=pd.DataFrame(data)
        df.replace(np.nan, 0.0, inplace=True)
        #print(df.columns)
        for i_ce in [in_para,]:
            df_CE_temp_0    = df.groupby(['Activity','isFinal',])[i_ce].apply(list)
            df_CE_ave_temp  = np.mean(df_CE_temp_0['CMA Audit',1])
            df_CE_temp_1    = df.groupby(['Activity','State','isFinal',])[i_ce].apply(list)
            nb_etat=0
            for etat in states_ls_2:
                comp_array3     = df_CE_ave_temp
                temp_array3     = df_CE_temp_1['CMA Audit',etat,1][0]
                input_matrix[nb_etat][nb_idx] = (temp_array3>=comp_array3)
                nb_etat+=1
            nb_idx+=1

year 2013
year 2014
year 2013
year 2014
year 2014
year 2014
year 2014
year 2014
year 2013
year 2014
year 2013
year 2014


In [6]:
### write csv file
#print(list(row))
def flattenList(data):
    results = []
    for rec in data:
        if isinstance(rec, list):
            results.extend(rec)
            results = flattenList(results)
        else:
            results.append(rec)
    return results
rowname =  flattenList(row)
print(rowname)

if i_train_input:
    print('\n Creating training data by: \n',states_ls_2)
    savedata = pd.DataFrame(input_matrix,index=states_ls_2)
    savedata.to_csv("EI_training_input_with_header.csv", header=rowname,index=states_ls_2)
    savedata.to_csv("EI_training_input.csv", header=False,index=False)
    savedata = pd.DataFrame(result_matrix,index=states_ls_2)
    savedata.to_csv("EI_training_output.csv", header=False,index=False)
    savedata.to_csv("EI_training_output_with_header.csv", header=False,index=states_ls_2)
if i_test_input:
    print('\n Creating validation data by: \n',states_ls_2)
    savedata = pd.DataFrame(input_matrix,index=states_ls_2)
    savedata.to_csv("EI_validation_input.csv", header=False,index=False)
    savedata = pd.DataFrame(result_matrix,index=states_ls_2)
    savedata.to_csv("EI_validation_output.csv", header=False,index=False)

if i_application_input:
    print('\n Creating application data by: \n',states_ls_2)
    savedata = pd.DataFrame(input_matrix,index=states_ls_2)
    savedata.to_csv("EI_application_input.csv", header=False,index=False)

['EI', 'dYear', 'GDP', 'ce6', 'ce3', 'ce2']

 Creating application data by: 
 ['PRK', 'TLS', 'COM', 'ZWE', 'TUN', 'TKM', 'OMN', 'SAU', 'BOL', 'COL', 'CIV', 'SEN']
